Installing dependencies

In [25]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [26]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [27]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import requests
from binance.client import Client

In [28]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']
ta_api = os.environ['taapi_key']


# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
# info = client.get_symbol_info('ETHUSDT')
# print(info)

In [29]:
def get_data_frame(symbol):
    starttime = '3 days ago UTC'
    interval = '30m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1667095200000,1615.32000000,1638.28000000,1229.00000000,1610.38000000
1,1667097000000,1610.44000000,1650.00000000,1229.00000000,1615.91000000
2,1667098800000,1616.64000000,1623.99000000,1229.00000000,1623.97000000
3,1667100600000,1624.05000000,1629.75000000,668.00000000,1622.16000000
4,1667102400000,1622.19000000,1646.60000000,1229.00000000,1623.06000000
...,...,...,...,...,...
139,1667345400000,1581.76000000,1628.23000000,1578.51000000,1578.51000000
140,1667347200000,1578.48000000,3000.00000000,1572.12000000,1580.73000000
141,1667349000000,1580.77000000,2000.00000000,1229.00000000,1587.49000000
142,1667350800000,1587.55000000,1619.28000000,1249.69000000,1590.37000000


In [30]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
0,2022-10-30 02:00:00,1615.32,1638.28,1229.00,1610.38,0.000000,0.000000,1610.380000,0
1,2022-10-30 02:30:00,1610.44,1650.00,1229.00,1615.91,0.441140,0.088228,1610.435025,1
2,2022-10-30 03:00:00,1616.64,1623.99,1229.00,1623.97,1.424698,0.355522,1610.569701,1
3,2022-10-30 03:30:00,1624.05,1629.75,668.00,1622.16,2.034668,0.691351,1610.685028,1
4,2022-10-30 04:00:00,1622.19,1646.60,1229.00,1623.06,2.561173,1.065315,1610.808162,1
...,...,...,...,...,...,...,...,...,...
139,2022-11-01 23:30:00,1581.76,1628.23,1578.51,1578.51,5.666602,6.571768,1583.249021,0
140,2022-11-02 00:00:00,1578.48,3000.00,1572.12,1580.73,5.559346,6.369283,1583.223956,0
141,2022-11-02 00:30:00,1580.77,2000.00,1229.00,1587.49,5.951219,6.285670,1583.266405,0
142,2022-11-02 01:00:00,1587.55,1619.28,1249.69,1590.37,6.420165,6.312569,1583.337087,1


# BACKTESTING

Backtesting completed on 6 month historical data on 5m time frame

In [31]:
def get_six_month_data_frame(symbol):
    starttime = '6 months ago UTC'
    interval = '30m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

In [32]:
six_month_eth_df_raw

,date,open,high,low,close
0,1664969400000,1334.55000000,1334.55000000,1331.21000000,1331.21000000
1,1664971200000,1334.19000000,4277.48000000,1069.37000000,4277.48000000
2,1664973000000,1331.64000000,4277.48000000,1328.61000000,1332.87000000
3,1664974800000,1332.30000000,1334.02000000,1322.05000000,1323.54000000
4,1664976600000,1323.52000000,1328.28000000,397.56000000,1323.65000000
...,...,...,...,...,...
1320,1667345400000,1581.76000000,1628.23000000,1578.51000000,1578.51000000
1321,1667347200000,1578.48000000,3000.00000000,1572.12000000,1580.73000000
1322,1667349000000,1580.77000000,2000.00000000,1229.00000000,1587.49000000
1323,1667350800000,1587.55000000,1619.28000000,1249.69000000,1590.37000000


In [54]:
def test_macd_logic(symbol):

    symbol_df = get_six_month_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)
    
    tr_list = [(symbol_df['high'] - symbol_df['low']), (symbol_df['low'] - symbol_df['close'].shift()), (symbol_df['high'] - symbol_df['close'].shift())]

    symbol_df['TR1'] = symbol_df['high'] - symbol_df['low']
    symbol_df['prev close'] = symbol_df['close'].shift()
    symbol_df['TR2'] = symbol_df['low'] - symbol_df['close'].shift()
    symbol_df['TR3'] = symbol_df['high'] - symbol_df['close'].shift()
    # symbol_df['TR LIST'] = symbol_df.apply(lambda row : [(row['high'] - row['low']), (row['low'] - row['close'].shift()), (row['high'] - row['close'].shift())])
    # symbol_df['TR LIST'] = symbol_df.apply(symbol_df)
    print(symbol_df)

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return symbol_df

test_df = test_macd_logic('ETHUSDT')

                    date     open     high      low    close        macd  \
0    2022-10-05 11:30:00  1334.55  1334.55  1331.21  1331.21    0.000000   
1    2022-10-05 12:00:00  1334.19  4277.48  1069.37  4277.48  235.030085   
2    2022-10-05 12:30:00  1331.64  4277.48  1328.61  1332.87  181.594396   
3    2022-10-05 13:00:00  1332.30  1334.02  1322.05  1323.54  136.915103   
4    2022-10-05 13:30:00  1323.52  1328.28   397.56  1323.65  100.358447   
...                  ...      ...      ...      ...      ...         ...   
1321 2022-11-02 00:00:00  1578.48  3000.00  1572.12  1580.73    5.559201   
1322 2022-11-02 00:30:00  1580.77  2000.00  1229.00  1587.49    5.951084   
1323 2022-11-02 01:00:00  1587.55  1619.28  1249.69  1590.37    6.420041   
1324 2022-11-02 01:30:00  1590.44  1591.06  1584.99  1586.08    6.372071   
1325 2022-11-02 02:00:00  1586.01  1619.28  1229.00  1582.42    5.969906   

         signal      200_ema  Trigger      TR1  prev close      TR2      TR3  
0      0

In [43]:
test_df

,date,open,high,low,close,macd,signal,200_ema,Trigger,TR1,prev close,TR2,TR3
200,2022-10-09 15:30:00,1324.56,1326.69,1323.70,1326.47,2.348542,1.760207,1335.455558,0,2.99,1324.71,-1.01,1.98
201,2022-10-09 16:00:00,1326.50,1327.64,1229.00,1326.16,2.404038,1.888973,1335.363065,0,98.64,1326.47,-97.47,1.17
202,2022-10-09 16:30:00,1325.72,1326.45,1324.34,1325.50,2.367472,1.984673,1335.264925,0,2.11,1326.16,-1.82,0.29
203,2022-10-09 17:00:00,1325.43,1326.00,1229.00,1324.22,2.209735,2.029685,1335.155025,0,97.00,1325.50,-96.50,0.50
204,2022-10-09 17:30:00,1324.25,1325.56,1323.09,1324.49,2.082509,2.040250,1335.048906,0,2.47,1324.22,-1.13,1.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,2022-11-02 00:00:00,1578.48,3000.00,1572.12,1580.73,5.559201,6.369070,1566.219561,0,1427.88,1578.51,-6.39,1421.49
1322,2022-11-02 00:30:00,1580.77,2000.00,1229.00,1587.49,5.951084,6.285473,1566.431207,0,771.00,1580.73,-351.73,419.27
1323,2022-11-02 01:00:00,1587.55,1619.28,1249.69,1590.37,6.420041,6.312386,1566.669404,0,369.59,1587.49,-337.80,31.79
1324,2022-11-02 01:30:00,1590.44,1591.06,1584.99,1586.08,6.372071,6.324323,1566.862545,0,6.07,1590.37,-5.38,0.69


In [35]:
atr_indicator = "atr"
atr_endpoint = f"https://api.taapi.io/{atr_indicator}"

live_price_indicator = "price"
live_price_endpoint = f"https://api.taapi.io/{atr_indicator}"

stop_loss = 0
tp = 0
entry = 0

parameters = {
    'secret': ta_api,
    'exchange': 'binance',
    'symbol': 'ETH/USDT',
    'interval': '30m'
    } 

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

atr_test = 16.74326363683623

for index, row in test_df.iterrows():
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True

            stop_loss = row['close'] - atr_test
            tp = row['close'] + atr_test
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr_test))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

STOP LOSS: 1280.7867363631638, CLOSE: 1220.0, ENTRY: 1297.53
Losses: 16.743263636836218

STOP LOSS: 1246.7467363631638, CLOSE: 1242.8, ENTRY: 1263.49
Losses: 33.486527273672436

STOP LOSS: 1297.3667363631637, CLOSE: 1295.94, ENTRY: 1314.11
Losses: 50.229790910508655

STOP LOSS: 1286.2367363631638, CLOSE: 1285.29, ENTRY: 1302.98
Losses: 66.97305454734487

TP1: 1327.4732636368362, CLOSE: 1328.23, ENTRY: 1310.73
Profit: 8.371631818418109

TP2: 1344.2165272736725, CLOSE: 1361.13, ENTRY: 1310.73
Profit: 25.114895455254327

TP1: 1363.6732636368363, CLOSE: 1364.74, ENTRY: 1346.93
Profit: 33.486527273672436

TP2: 1380.4165272736725, CLOSE: 1381.75, ENTRY: 1346.93
Profit: 50.229790910508655

STOP LOSS: 1538.7367363631638, CLOSE: 1516.77, ENTRY: 1555.48
Losses: 83.71631818418109

STOP LOSS: 1499.3067363631637, CLOSE: 1493.55, ENTRY: 1516.05
Losses: 100.45958182101731

TP1: 1525.0832636368361, CLOSE: 1531.63, ENTRY: 1508.34
Profit: 58.601422728926764

TP2: 1541.8265272736724, CLOSE: 1545.61, ENTR

In [58]:
eth_raw_df_2 = get_six_month_data_frame('ETHUSDT')
eth_raw_df_2

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.04000000,1555.87000000,1555.87000000
1,1667395800000,1556.01000000,1557.38000000,1414.70000000,1553.45000000
2,1667397600000,1553.25000000,1572.20000000,1540.44000000,1558.06000000
3,1667399400000,1557.93000000,1562.92000000,1523.86000000,1560.21000000
4,1667401200000,1560.22000000,1560.65000000,1481.81000000,1558.80000000
5,1667403000000,1558.91000000,1561.57000000,1500.96000000,1558.54000000
6,1667404800000,1558.51000000,1559.79000000,1553.78000000,1553.92000000
7,1667406600000,1553.92000000,1574.85000000,1550.09000000,1571.23000000
8,1667408400000,1570.95000000,1572.09000000,1556.75000000,1568.18000000
9,1667410200000,1568.26000000,1577.10000000,1550.91000000,1558.57000000


In [104]:
def test_logic_2(raw_df):
    raw_df['open'] = pd.to_numeric(raw_df['open'])
    raw_df['high'] = pd.to_numeric(raw_df['high'])
    raw_df['low'] = pd.to_numeric(raw_df['low'])
    raw_df['close'] = pd.to_numeric(raw_df['close'])
    raw_df['date'] = pd.to_datetime(raw_df['date'], unit = 'ms')


    short_ema = raw_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = raw_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = raw_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    raw_df['macd'] = macd
    raw_df['signal'] = signal
    raw_df['200_ema'] = ema_200

    raw_df['TR1'] = abs(raw_df['high'] - raw_df['low'])
    raw_df['TR2'] = abs(raw_df['low'] - raw_df['close'].shift())
    raw_df['TR3'] = abs(raw_df['high'] - raw_df['close'].shift())

    raw_df['ATR'] = raw_df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    return raw_df

def row_check(TR1, TR2, TR3):
    if TR1 >= TR2 and TR1 >= TR3:
        ATR = TR1
    elif TR2 >= TR1 and TR2 >= TR3:
        ATR = TR2
    else:
        ATR = TR3
    
    return ATR

In [105]:
tinkered_df = test_logic_2(eth_raw_df_2)

In [106]:
tinkered_df

,date,open,high,low,close,macd,signal,200_ema,TR1,TR2,TR3,ATR
0,2022-11-02 13:00:00,1556.31,1557.04,1555.87,1555.87,0.000000,0.000000,1555.870000,1.17,NaN,NaN,NaN
1,2022-11-02 13:30:00,1556.01,1557.38,1414.70,1553.45,-0.193048,-0.038610,1555.845920,142.68,141.17,1.51,142.68
2,2022-11-02 14:00:00,1553.25,1572.20,1540.44,1558.06,0.025652,-0.025757,1555.867951,31.76,13.01,18.75,31.76
3,2022-11-02 14:30:00,1557.93,1562.92,1523.86,1560.21,0.368216,0.053037,1555.911156,39.06,34.20,4.86,39.06
4,2022-11-02 15:00:00,1560.22,1560.65,1481.81,1558.80,0.519932,0.146416,1555.939900,78.84,78.40,0.44,78.84
5,2022-11-02 15:30:00,1558.91,1561.57,1500.96,1558.54,0.612132,0.239559,1555.965772,60.61,57.84,2.77,60.61
6,2022-11-02 16:00:00,1558.51,1559.79,1553.78,1553.92,0.308845,0.253417,1555.945416,6.01,4.76,1.25,6.01
7,2022-11-02 16:30:00,1553.92,1574.85,1550.09,1571.23,1.448563,0.492446,1556.097501,24.76,3.83,20.93,24.76
8,2022-11-02 17:00:00,1570.95,1572.09,1556.75,1568.18,2.081691,0.810295,1556.217725,15.34,14.48,0.86,15.34
9,2022-11-02 17:30:00,1568.26,1577.10,1550.91,1558.57,1.787399,1.005716,1556.241131,26.19,17.27,8.92,26.19


In [36]:
# indicator = "atr"
# endpoint = f"https://api.taapi.io/{indicator}"
# parameters = {
#     'secret': ta_api,
#     'exchange': 'binance',
#     'symbol': 'ETH/USDT',
#     'interval': '15m'
#     } 
# response = requests.get(url = endpoint, params = parameters)
result = response.json() 
print(result['value'])

NameError: name 'response' is not defined